In [1]:
from carbon.helpers.stdimports import *
from carbon import CarbonOrderUI
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonOrderUI))
print_version(require="2.3.3")

[stdimports] imported np, pd, plt, os, sqrt, exp, log
CarbonOrderUI v1.9 (14/Mar/2023)
Carbon v2.3.3-BETA6 (11/Mar/2023)


# CarbonOrderUI (NBTest 062)

We introduced new properties that allow getting more curve parameters such as $x_0$ and $x_{asym}$. The code below introduces those parameters and at the same time asserts key relationships.

## Selling the base token

In [2]:
o = CarbonOrderUI.from_prices(
    pair="ETH/USDC", 
    tkn="ETH", 
    pa=2500, 
    pb=3000, 
    yint=10, 
    y=5)
o

CarbonOrderUI(pair=P('ETH/USDC'), tkn=ETH, B=0.018257418583505537, S=0.001742581416494464, yint=10, y=5, id=None, linked=None)

### Prices

Below the different ways of accessing the prices. `pa=py=p_start` is always the beginning of the range, and `pb=px=p_end` is always the end of the range. `pmax` and `pmin` are always the higher and lower price in the quote convention of the pair, respectively. Those figures are quoted in the convention of the pair, so as we are selling the base token this is the inverse of the internal quotation $dy/dx$ which is `pa_raw` and `pb_raw`. `p0` and `p0_raw` are is the geometric average of the start and end prices in the respective quotation. `reverseq` is true if and only if the `_raw` figures are the inverse of the other figures.

In [3]:
print("p_start", o.p_start)
print("p_end", o.p_end)
print("p0", o.p0)
print("reverseq", o.reverseq)
print("pa_raw", o.pa_raw)
print("pb_raw", o.pb_raw)
print("p0_raw", o.p0_raw)
print("p_min", o.pmin)
print("p_max", o.pmax)
assert abs(o.p_start/2500-1)<1e-10
assert abs(o.p_end/3000-1)<1e-10
assert o.pa is o.p_start
assert o.pb is o.p_end
assert o.py is o.pa
assert o.px is o.pb
assert o.pmin == min(o.p_start, o.p_end)
assert o.pmax == max(o.p_start, o.p_end)
assert o.pa_raw == 1/o.pa if o.reverseq else o.pa
assert o.pb_raw == 1/o.pb if o.reverseq else o.pb
assert abs(o.p0/sqrt(o.pa*o.pb)-1)<1e-10
assert abs(o.p0_raw/sqrt(o.pa_raw*o.pb_raw)-1)<1e-10

p_start 2500.0
p_end 3000.0
p0 2738.6127875258308
reverseq True
pa_raw 0.0004
pb_raw 0.0003333333333333333
p0_raw 0.0003651483716701108
p_min 2500.0
p_max 3000.0


### Curve capacity

`yint` is the curve capacity in the token being sold (here: ETH), and `xint` the corresponding sale amount. `y0` and `x0` are reference parameters determining the curve, and we have `yint/xint=y0/x0=p0_raw`

In [4]:
print("yint", o.yint)
print("xint", o.xint)
print("y0", o.y0)
print("x0", o.x0)
assert o.yint == 10
assert abs(o.yint/o.xint/o.p0_raw-1)<1e-10
assert o.y0/o.x0 == o.p0_raw

yint 10
xint 27386.127875258306
y0 4.886068751350676
x0 13381.050363179327


### Curve convexity

The curve convexity determines how different `pa` and `pb` are. There are a number of ultimately equivalent paramters that describe that

- `widthpc` is the percentage width, defined as `(pa_raw-pb_raw)/p0_raw`
- `widthr` is the width ratio, defined as `pa_raw/pb_raw >= 1`
- `Gamma` is the curve leverage parameter between `0..1` where `0` corresponds to constant price and `1` corresponds to constant product
- `Q` is an alternative convexity parameter also between `0..1` but the other way

The relationshipt between the parameters is provided in the functions `Gamma_from_Q`

$$
\Gamma(Q) = 1 - \sqrt{Q}
$$ 

and `Q_from_Gamma` 

$$
Q(\Gamma) = (1-\Gamma)^2
$$

Also `Q=1/sqrt(widthr)` because

$$
Q = \sqrt{\frac{p_a}{p_b}}
$$

In [5]:
print("widthpc", o.widthpc)
print("widthr", o.widthr)
print("Gamma", o.Gamma)
print("Q", o.Q)
assert abs(o.widthpc/((o.pa_raw-o.pb_raw)/o.p0_raw)-1)<1e-10
assert abs(o.widthr/(o.pa_raw / o.pb_raw)-1)<1e-10
assert abs(o.widthr/(o.pmax / o.pmin)-1)<1e-10
assert abs(o.Q/(sqrt(1/o.widthr))-1)<1e-10
assert abs(o.Q/o.Q_from_Gamma(o.Gamma)-1)<1e-10
assert abs(o.Gamma/o.Gamma_from_Q(o.Q)-1)<1e-10

widthpc 0.18257418583505536
widthr 1.2
Gamma 0.044557207795633214
Q 0.9128709291752768


### Asymptotes

Here we deal with the curve asymptotes `yasym` and `xasym`. They are related to the other token figures via 

$$
\frac{x_{asym}}{x_0} = \frac{y_{asym}}{y_0} = 1-\frac 1 \Gamma
$$

We also look at the relationship between the `int`ercepts and the `0` figures which are

$$
\frac{x_{int}}{x_0} = \frac{y_{int}}{y_0} = \frac {2-\Gamma} {1-\Gamma}
$$

In [6]:
print("yasym", o.yasym)
print("xasym", o.xasym)
print("yint", o.yint)
print("xint", o.xint)
print("y0", o.y0)
print("x0", o.x0)
assert abs(o.xasym/o.x0/(o.yasym/o.y0)-1)<1e-10
assert abs(o.yasym/o.y0/o.asym_over_0(o.Gamma)-1)<1e-10
assert abs(o.xasym/o.x0/o.asym_over_0(o.Gamma)-1)<1e-10
assert abs(o.xint/o.x0/(o.yint/o.y0)-1)<1e-10
assert abs(o.yint/o.y0/o.int_over_0(o.Gamma)-1)<1e-10
assert abs(o.xint/o.x0/o.int_over_0(o.Gamma)-1)<1e-10

yasym -104.77225575051658
xasym -286930.6393762914
yint 10
xint 27386.127875258306
y0 4.886068751350676
x0 13381.050363179327


### Kappa and the invariant function

A very elegant way to express the invariant function is 

$$
(x-x_{asym})(y-y_{asym}) = \kappa = \frac{x_0y_0}{\Gamma^2}
$$

or in way that scales better as

$$
\sqrt{(x-x_{asym})(y-y_{asym})} = \bar\kappa = \frac{\sqrt{x_0y_0}}{\Gamma}
$$


In [7]:
print("y0", o.y0)
print("x0", o.x0)
print("Gamma", o.Gamma)
print("leverage_fctr", o.leverage_fctr)
print("kappa", o.kappa)
print("kappa_bar", o.kappa_bar)
assert o.leverage_fctr == 1/o.Gamma
assert abs(o.kappa/(o.x0*o.y0/o.Gamma**2)-1)<1e-10
assert abs(o.kappa_bar/(sqrt(o.x0*o.y0)/o.Gamma)-1)<1e-10

y0 4.886068751350676
x0 13381.050363179327
Gamma 0.044557207795633214
leverage_fctr 22.44305802523838
kappa 32931676.725154955
kappa_bar 5738.612787525828


Here we are checking the invariant function against the expression above. We are comparing both against `xfromy_f` and `yfromx_f` where we recall that `y` is the token being sold and `x` the token being bought by the AMM.

In [8]:
for y in [0.1,1,3,5,7,9,9.99]:
    x = o.xfromy_f(y)
    f2 = (x-o.xasym)*(y-o.yasym)
    f = sqrt(f2)
    print(f"y={y} ->", x, f, o.kappa_bar, f2, o.kappa)
    assert abs(f/o.kappa_bar-1) < 1e-10
    assert abs(f2/o.kappa-1) < 1e-10

y=0.1 -> 27086.413937570833 5738.612787525828 5738.612787525828 32931676.72515496 32931676.725154955
y=1 -> 24414.490700450162 5738.612787525828 5738.612787525828 32931676.72515496 32931676.725154955
y=3 -> 18636.656175069555 5738.612787525829 5738.612787525828 32931676.725154962 32931676.725154955
y=5 -> 13069.360623708471 5738.612787525829 5738.612787525828 32931676.725154962 32931676.725154955
y=7 -> 7701.3021912183785 5738.612787525828 5738.612787525828 32931676.72515496 32931676.725154955
y=9 -> 2521.9737227104124 5738.612787525828 5738.612787525828 32931676.72515496 32931676.725154955
y=9.99 -> 25.002178416573496 5738.612787525829 5738.612787525828 32931676.725154962 32931676.725154955


## Selling the quote token

In [9]:
o = CarbonOrderUI.from_prices(
    pair="ETH/USDC", 
    tkn="USDC", 
    pa=1500, 
    pb=1000, 
    yint=3000, 
    y=1500)
o

CarbonOrderUI(pair=P('ETH/USDC'), tkn=USDC, B=31.622776601683793, S=7.107056860390376, yint=3000, y=1500, id=None, linked=None)

### Prices

In [10]:
print("p_start", o.p_start)
print("p_end", o.p_end)
print("p0", o.p0)
print("reverseq", o.reverseq)
print("pa_raw", o.pa_raw)
print("pb_raw", o.pb_raw)
print("p0_raw", o.p0_raw)
print("p_min", o.pmin)
print("p_max", o.pmax)
assert abs(o.p_start/1500-1)<1e-10
assert abs(o.p_end/1000-1)<1e-10
assert o.pa is o.p_start
assert o.pb is o.p_end
assert o.py is o.pa
assert o.px is o.pb
assert o.pmin == min(o.p_start, o.p_end)
assert o.pmax == max(o.p_start, o.p_end)
assert o.pa_raw == 1/o.pa if o.reverseq else o.pa
assert o.pb_raw == 1/o.pb if o.reverseq else o.pb
assert abs(o.p0/sqrt(o.pa*o.pb)-1)<1e-10
assert abs(o.p0_raw/sqrt(o.pa_raw*o.pb_raw)-1)<1e-10

p_start 1500.0
p_end 1000.0
p0 1224.744871391589
reverseq False
pa_raw 1500.0
pb_raw 1000.0
p0_raw 1224.744871391589
p_min 1000.0
p_max 1500.0


### Curve capacity


In [11]:
print("yint", o.yint)
print("xint", o.xint)
print("y0", o.y0)
print("x0", o.x0)
assert o.yint == 3000
assert abs(o.yint/o.xint/o.p0_raw-1)<1e-10
assert o.y0/o.x0 == o.p0_raw

yint 3000
xint 2.4494897427831783
y0 1424.0403223409987
x0 1.1627240542946424


### Curve convexity

In [12]:
print("widthpc", o.widthpc)
print("widthr", o.widthr)
print("Gamma", o.Gamma)
print("Q", o.Q)
assert abs(o.widthpc/((o.pa_raw-o.pb_raw)/o.p0_raw)-1)<1e-10
assert abs(o.widthr/(o.pa_raw / o.pb_raw)-1)<1e-10
assert abs(o.widthr/(o.pmax / o.pmin)-1)<1e-10
assert abs(o.Q/(sqrt(1/o.widthr))-1)<1e-10
assert abs(o.Q/o.Q_from_Gamma(o.Gamma)-1)<1e-10
assert abs(o.Gamma/o.Gamma_from_Q(o.Q)-1)<1e-10

widthpc 0.4082482904638631
widthr 1.5
Gamma 0.09639799639015512
Q 0.816496580927726


### Asymptotes

In [13]:
print("yasym", o.yasym)
print("xasym", o.xasym)
print("yint", o.yint)
print("xint", o.xint)
print("y0", o.y0)
print("x0", o.x0)
assert abs(o.xasym/o.x0/(o.yasym/o.y0)-1)<1e-10
assert abs(o.yasym/o.y0/o.asym_over_0(o.Gamma)-1)<1e-10
assert abs(o.xasym/o.x0/o.asym_over_0(o.Gamma)-1)<1e-10
assert abs(o.xint/o.x0/(o.yint/o.y0)-1)<1e-10
assert abs(o.yint/o.y0/o.int_over_0(o.Gamma)-1)<1e-10
assert abs(o.xint/o.x0/o.int_over_0(o.Gamma)-1)<1e-10

yasym -13348.469228349542
xasym -10.898979485566363
yint 3000
xint 2.4494897427831783
y0 1424.0403223409987
x0 1.1627240542946424


### Kappa and invariant

In [14]:
print("y0", o.y0)
print("x0", o.x0)
print("Gamma", o.Gamma)
print("leverage_fctr", o.leverage_fctr)
print("kappa", o.kappa)
print("kappa_bar", o.kappa_bar)
assert o.leverage_fctr == 1/o.Gamma
assert abs(o.kappa/(o.x0*o.y0/o.Gamma**2)-1)<1e-10
assert abs(o.kappa_bar/(sqrt(o.x0*o.y0)/o.Gamma)-1)<1e-10

y0 1424.0403223409987
x0 1.1627240542946424
Gamma 0.09639799639015512
leverage_fctr 10.373659593013361
kappa 178181.6307401946
kappa_bar 422.115660382548


Here we are checking the invariant function against the expression above. We are comparing both against `xfromy_f` and `yfromx_f` where we recall that `y` is the token being sold and `x` the token being bought by the AMM.

In [15]:
for y in [1,100,1000,1500,2500,2999]:
    x = o.xfromy_f(y)
    f2 = (x-o.xasym)*(y-o.yasym)
    f = sqrt(f2)
    print(f"y={y} ->", x, f, o.kappa_bar, f2, o.kappa)
    assert abs(f/o.kappa_bar-1) < 1e-10
    assert abs(f2/o.kappa-1) < 1e-10

y=1 -> 2.448489817692524 422.115660382548 422.115660382548 178181.6307401946 178181.6307401946
y=100 -> 2.3502333218352027 422.115660382548 422.115660382548 178181.6307401946 178181.6307401946
y=1000 -> 1.5191835884530849 422.115660382548 422.115660382548 178181.6307401946 178181.6307401946
y=1500 -> 1.1010205144336438 422.115660382548 422.115660382548 178181.6307401946 178181.6307401946
y=2500 -> 0.3438495960881321 422.115660382548 422.115660382548 178181.6307401946 178181.6307401946
y=2999 -> 0.0006667074476987246 422.115660382548 422.115660382548 178181.6307401946 178181.6307401946
